In [40]:
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
import numpy as np

In [18]:
df = pd.read_csv("../../data/mturks_results.csv")
df.columns = ['ingredients', 'preparation', 'description','relatedRecipes', 'current_url', 'document_url','first_annotator', 'second_annotator','third_annotator','most_common']
df = df[df['most_common'] != "None"]
le = LabelEncoder()
df['target'] = le.fit_transform(df.most_common)
df.dropna(inplace=True)
df['text'] = df[['ingredients', 'preparation', 'description', 'document_url']].agg(' '.join, axis=1)

In [19]:
classes_dict = dict(zip(df.target, df.most_common))
classes_dict

{0: 'Appetizer or side dish',
 1: 'Dessert',
 3: 'Main course',
 2: 'Drinks',
 4: 'Other'}

In [63]:
preprocessor = CountVectorizer(max_features=2000)

In [66]:
# Train on whole dataset
tfidf_transformer = TfidfTransformer()
processed = preprocessor.fit_transform(df['text'].to_list())
train_tfidf = tfidf_transformer.fit_transform(processed)
clf = LinearSVC(class_weight="balanced").fit(train_tfidf, df['target'].to_list())

In [67]:
clf.score(train_tfidf, df['target'])

0.9741496598639455

(735, 1)

In [71]:
# Save model, transformers to pickle

import pickle

filehandler = open("linearSVC_model.pkl","wb")
pickle.dump(clf,filehandler)
filehandler.close()

filehandler = open("linearSVC_preprocessor.pkl","wb")
pickle.dump(preprocessor,filehandler)
filehandler.close()

filehandler = open("linearSVC_tfidfTransformer.pkl","wb")
pickle.dump(tfidf_transformer,filehandler)
filehandler.close()

In [85]:
import pickle
import numpy as np

class LinearClassifier:
    def __init__(self):
        self.model = pickle.load(open("linearSVC_model.pkl", 'rb'))
        self.preprocessor = pickle.load(open("linearSVC_preprocessor.pkl", 'rb'))
        self.transformer = pickle.load(open("linearSVC_tfidfTransformer.pkl", 'rb'))
        self.classes = {0: 'Appetizer or side dish', 1: 'Dessert', 3: 'Main course', 2: 'Drinks', 4: 'Other'}

    def predict(self, text):
        processed = self.preprocessor.transform(text)
        transformed = self.transformer.transform(processed)
        prediction = self.model.predict(transformed)[0]
        return self.classes[prediction]

In [86]:
classifier = LinearClassifier()

In [87]:
classifier.predict(["text"])

'Appetizer or side dish'